# VAE Tests

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from ocpa.objects.log.importer.ocel import factory as ocel_import_factory
from ocpa.algo.discovery.ocpn import algorithm as ocpn_discovery_factory
from src.utils import get_happy_path_log, create_flower_model, generate_variant_model, sample_traces

In [18]:
from tensorflow.python.framework.ops import disable_eager_execution
disable_eager_execution()

In [3]:
import argparse
import json
from keras import backend as K
from keras.losses import categorical_crossentropy
from keras.layers import Layer
from keras.layers import Input, LSTM, TimeDistributed
from keras.layers.core import Dense, Lambda
from keras.models import Model
from nltk.tokenize import word_tokenize
import tensorflow as tf
import pandas as pd
import numpy as np
import codecs
import os

In [4]:
filename = "../src/data/jsonocel/order_process.jsonocel"
ocel = ocel_import_factory.apply(filename)
ocpn = ocpn_discovery_factory.apply(ocel, parameters={"debug": False})
train_log = sample_traces(ocel, ocpn, 10000)


Generate the traces: 100%|██████████| 10000/10000 [00:01<00:00, 5944.62it/s]


In [5]:
train_log

[['Fuel Car',
  'Place Order',
  'Confirm Order',
  'Pick Item',
  'Load Cargo',
  'Item out of stock',
  'Pay Order',
  'Start Route',
  'Reorder Item',
  'End Route'],
 ['Place Order',
  'Confirm Order',
  'Pay Order',
  'Pick Item',
  'Item out of stock',
  'Reorder Item',
  'Fuel Car',
  'Load Cargo',
  'Start Route',
  'End Route'],
 ['Fuel Car',
  'Place Order',
  'Confirm Order',
  'Payment Reminder',
  'Payment Reminder',
  'Payment Reminder',
  'Payment Reminder',
  'Pay Order',
  'Item out of stock',
  'Reorder Item',
  'Pick Item',
  'Load Cargo',
  'Start Route',
  'End Route'],
 ['Place Order',
  'Confirm Order',
  'Fuel Car',
  'Item out of stock',
  'Reorder Item',
  'Pay Order',
  'Pick Item',
  'Load Cargo',
  'Start Route',
  'End Route'],
 ['Place Order',
  'Confirm Order',
  'Payment Reminder',
  'Fuel Car',
  'Payment Reminder',
  'Payment Reminder',
  'Pay Order',
  'Pick Item',
  'Item out of stock',
  'Reorder Item',
  'Pick Item',
  'Load Cargo',
  'Start Route

In [6]:
data_path = '../src/data/sampled_data_order.txt'

In [7]:
with open(data_path, "w", encoding="utf-8") as file:
    for sentence in train_log:
        line = " ".join(sentence) + "\n"
        file.write(line)

In [12]:
def create_lstm_vae(input_dim,
                    batch_size,  # we need it for sampling
                    intermediate_dim,
                    latent_dim):
    """
    Creates an LSTM Variational Autoencoder (VAE).

    # Arguments
        input_dim: int.
        batch_size: int.
        intermediate_dim: int, output shape of LSTM.
        latent_dim: int, latent z-layer shape.
        epsilon_std: float, z-layer sigma.


    # References
        - [Building Autoencoders in Keras](https://blog.keras.io/building-autoencoders-in-keras.html)
        - [Generating sentences from a continuous space](https://arxiv.org/abs/1511.06349)
    """
    x = Input(shape=(None, input_dim,))

    # LSTM encoding
    h = LSTM(units=intermediate_dim)(x)

    # VAE Z layer
    z_mean = Dense(units=latent_dim)(h)
    z_log_sigma = Dense(units=latent_dim)(h)

    def sampling(args):
        z_mean, z_log_sigma = args
        epsilon = K.random_normal(shape=(batch_size, latent_dim), mean=0., stddev=1.0)
        return z_mean + z_log_sigma * epsilon

    # note that "output_shape" isn't necessary with the TensorFlow backend
    # so you could write `Lambda(sampling)([z_mean, z_log_sigma])`
    z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_sigma])

    z_reweighting = Dense(units=intermediate_dim, activation="linear")
    z_reweighted = z_reweighting(z)

    # "next-word" data for prediction
    decoder_words_input = Input(shape=(None, input_dim,))

    # decoded LSTM layer
    decoder_h = LSTM(intermediate_dim, return_sequences=True, return_state=True)

    # todo: not sure if this initialization is correct
    h_decoded, _, _ = decoder_h(decoder_words_input, initial_state=[z_reweighted, z_reweighted])
    decoder_dense = TimeDistributed(Dense(input_dim, activation="softmax"))
    decoded_onehot = decoder_dense(h_decoded)

    # end-to-end autoencoder
    vae = Model([x, decoder_words_input], decoded_onehot)

    # encoder, from inputs to latent space
    encoder = Model(x, [z_mean, z_log_sigma])

    # generator, from latent space to reconstructed inputs -- for inference's first step
    decoder_state_input = Input(shape=(latent_dim,))
    _z_rewighted = z_reweighting(decoder_state_input)
    _h_decoded, _decoded_h, _decoded_c = decoder_h(decoder_words_input, initial_state=[_z_rewighted, _z_rewighted])
    _decoded_onehot = decoder_dense(_h_decoded)
    generator = Model([decoder_words_input, decoder_state_input], [_decoded_onehot, _decoded_h, _decoded_c])

    # RNN for inference
    input_h = Input(shape=(intermediate_dim,))
    input_c = Input(shape=(intermediate_dim,))
    __h_decoded, __decoded_h, __decoded_c = decoder_h(decoder_words_input, initial_state=[input_h, input_c])
    __decoded_onehot = decoder_dense(__h_decoded)
    stepper = Model([decoder_words_input, input_h, input_c], [__decoded_onehot, __decoded_h, __decoded_c])

    def vae_loss(x, x_decoded_onehot):
        xent_loss = categorical_crossentropy(x, x_decoded_onehot)
        kl_loss = - 0.5 * K.mean(1 + z_log_sigma - K.square(z_mean) - K.exp(z_log_sigma))
        loss = xent_loss + kl_loss
        return loss

    vae.compile(optimizer="adam", loss=vae_loss)
    vae.summary()

    return vae, encoder, generator, stepper

In [13]:
def decode_sequence(states_value, decoder_adapter_model, rnn_decoder_model, num_decoder_tokens, token2id, id2token, max_seq_length):
    """
    Decoding adapted from this example:
    https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html

    :param states_value:
    :param decoder_adapter_model: reads text representation, makes the first prediction, yields states after the first RNN's step
    :param rnn_decoder_model: reads previous states and makes one RNN step
    :param num_decoder_tokens:
    :param token2id: dict mapping words to ids
    :param id2token: dict mapping ids to words
    :param max_seq_length: the maximum length of the sequence
    :return:
    """

    # generate empty target sequence of length 1
    target_seq = np.zeros((1, 1, num_decoder_tokens))

    # populate the first token of the target sequence with the start character
    target_seq[0, 0, token2id["\t"]] = 1.0

    # sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1)
    stop_condition = False
    decoded_sentence = ""

    first_time = True
    h, c = None, None

    while not stop_condition:

        if first_time:
            # feeding in states sampled with the mean and std provided by encoder
            # and getting current LSTM states to feed in to the decoder at the next step
            output_tokens, h, c = decoder_adapter_model.predict([target_seq, states_value])
            first_time = False
        else:
            # reading output token
            output_tokens, h, c = rnn_decoder_model.predict([target_seq, h, c])

        # sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_token = id2token[sampled_token_index]
        decoded_sentence += sampled_token + " "

        # exit condition: either hit max length
        # or find stop character.
        if sampled_token == "<end>" or len(decoded_sentence) > max_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

    return decoded_sentence

In [14]:
def get_vectors(word2id={}, vectors_path="../src/data/glove.6B.50d.txt"):

    encoder_input_data = np.random.random((len(word2id), 50))

    for line in open(vectors_path):
        line = line.strip().split(" ")
        if line[0] in word2id:
            encoder_input_data[word2id[line[0]]] = np.array([float(n) for n in line[1:]])

    return encoder_input_data

In [15]:
def get_text_data(data_path, num_samples=1000):

    # vectorize the data
    input_texts = []
    input_characters = set(["\t"])

    with open(data_path, "r", encoding="utf-8") as f:
        lines = f.read().lower().split("\n")

    for line in lines[: min(num_samples, len(lines) - 1)]:

        #input_text, _ = line.split("\t")
        input_text = word_tokenize(line)
        input_text.append("<end>")

        input_texts.append(input_text)

        for char in input_text:
            if char not in input_characters:
                input_characters.add(char)

    input_characters = sorted(list(input_characters))
    num_encoder_tokens = len(input_characters)
    max_encoder_seq_length = max([len(txt) for txt in input_texts]) + 1

    print("Number of samples:", len(input_texts))
    print("Number of unique input tokens:", num_encoder_tokens)
    print("Max sequence length for inputs:", max_encoder_seq_length)

    input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
    reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())

    encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32")
    decoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32")

    for i, input_text in enumerate(input_texts):
        decoder_input_data[i, 0, input_token_index["\t"]] = 1.0

        for t, char in enumerate(input_text):
            encoder_input_data[i, t, input_token_index[char]] = 1.0
            decoder_input_data[i, t + 1, input_token_index[char]] = 1.0

    return max_encoder_seq_length, num_encoder_tokens, input_characters, input_token_index, reverse_input_char_index, \
           encoder_input_data, decoder_input_data

In [16]:
#     from argparse import ArgumentParser

#     p = ArgumentParser()
#     p.add_argument("--input", default="data/fra.txt", type=str)
#     p.add_argument("--num_samples", default=3000, type=int)
#     p.add_argument("--batch_size", default=1, type=int)
#     p.add_argument("--epochs", default=40, type=int)
#     p.add_argument("--latent_dim", default=191, type=int)
#     p.add_argument("--inter_dim", default=353, type=int)
#     p.add_argument("--samples", default=5, type=int)
#     args = p.parse_args()

timesteps_max, enc_tokens, characters, char2id, id2char, x, x_decoder = get_text_data(num_samples=1000,
                                                                                      data_path='../src/data/sampled_data_order.txt')

print(x.shape, "Creating model...")

Number of samples: 1000
Number of unique input tokens: 21
Max sequence length for inputs: 48
(1000, 48, 21) Creating model...


In [19]:
input_dim, timesteps = x.shape[-1], x.shape[-2]
batch_size, latent_dim = 1, 191
intermediate_dim, epochs = 353, 40

vae, enc, gen, stepper = create_lstm_vae(input_dim,
                                         batch_size=batch_size,
                                         intermediate_dim=intermediate_dim,
                                         latent_dim=latent_dim,
                                        )
print("Training model...")

vae.fit([x, x_decoder], x, epochs=epochs, verbose=1)

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, 21)]   0           []                               
                                                                                                  
 lstm_2 (LSTM)                  (None, 353)          529500      ['input_1[0][0]']                
                                                                                                  
 dense_4 (Dense)                (None, 191)          67614       ['lstm_2[0][0]']                 
                                                                                                  
 dense_5 (Dense)                (None, 191)          67614       ['lstm_2[0][0]']                 
                                                                                            

In [30]:
print("Fitted, predicting...")

def decode(s):
    return decode_sequence(s, gen, stepper, input_dim, char2id, id2char, timesteps_max)

log = []

for _ in range(10):

    id_from = np.random.randint(0, x.shape[0] - 1)
    id_to = np.random.randint(0, x.shape[0] - 1)

    m_from, std_from = enc.predict([[x[id_from]]])
    m_to, std_to = enc.predict([[x[id_to]]])

    seq_from = np.random.normal(size=(latent_dim,))
    seq_from = m_from + std_from * seq_from

    seq_to = np.random.normal(size=(latent_dim,))
    seq_to = m_to + std_to * seq_to

    print("==  \t", " ".join([id2char[j] for j in np.argmax(x[id_from], axis=1)]), "==")

    for v in np.linspace(0, 1, 7):
        print("%.2f\t" % (1 - v), decode(v * seq_to + (1 - v) * seq_from))

    print("==  \t", " ".join([id2char[j] for j in np.argmax(x[id_to], axis=1)]), "==")
    parts = " ".join([id2char[j] for j in np.argmax(x[id_to], axis=1)]).split(' <end>')
    log.append([parts[0]])

Fitted, predicting...
==  	 place order confirm order payment reminder fuel car payment reminder pick item pay order load cargo item out of stock reorder item start route pick item load cargo end route <end> 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 ==
1.00	 place order confirm order fuel car payment reminder 
0.83	 place order fuel car confirm order payment reminder 
0.67	 place order fuel car confirm order payment reminder 
0.50	 fuel car place order confirm order pay order pick 
0.33	 fuel car place order confirm order pick item pay 
0.17	 fuel car place order confirm order payment reminder 
0.00	 fuel car place order confirm order payment reminder 
==  	 fuel car place order confirm order payment reminder pay order item out of stock reorder item pick item load cargo start route end route <end> 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 ==
==  	 place order fuel car confirm order payment reminder pick item item out of stock reorder item pick item pay order load cargo start route end rout

In [31]:
log

[['fuel car place order confirm order payment reminder pay order item out of stock reorder item pick item load cargo start route end route'],
 ['fuel car place order confirm order pay order pick item item out of stock reorder item load cargo pick item load cargo start route end route'],
 ['place order fuel car confirm order item out of stock pick item load cargo pay order start route end route'],
 ['fuel car place order confirm order pick item payment reminder load cargo start route item out of stock pay order end route'],
 ['fuel car place order confirm order item out of stock payment reminder pick item load cargo start route end route payment reminder payment reminder payment reminder payment reminder payment reminder pay order'],
 ['place order confirm order fuel car pay order pick item load cargo item out of stock start route reorder item pick item end route'],
 ['place order fuel car confirm order pay order pick item item out of stock reorder item load cargo pick item load cargo s

In [7]:
GLOVE_EMBEDDING = '../src/data/glove.6B.300d.txt'
VALIDATION_SPLIT = 0.2
MAX_SEQUENCE_LENGTH = max(len(lst) for lst in train_log)
MAX_NB_WORDS = 1000
EMBEDDING_DIM = 300

In [8]:
# texts = [] 
# with codecs.open(TRAIN_DATA_FILE, encoding='utf-8') as f:
#     reader = csv.reader(f, delimiter=',')
#     header = next(reader)
#     for values in reader:
#         if len(values[3].split()) <= MAX_SEQUENCE_LENGTH:
#             texts.append(values[3])
#         if len(values[4].split()) <= MAX_SEQUENCE_LENGTH:
#             texts.append(values[4])
# print('Found %s texts in train.csv' % len(texts))
n_sents = len(texts)


In [9]:
#======================== Tokenize and pad texts lists ===================#
tokenizer = Tokenizer(MAX_NB_WORDS+1, oov_token='unk') #+1 for 'unk' token
tokenizer.fit_on_texts(texts)
print('Found %s unique tokens' % len(tokenizer.word_index))
## **Key Step** to make it work correctly otherwise drops OOV tokens anyway!
tokenizer.word_index = {e:i for e,i in tokenizer.word_index.items() if i <= MAX_NB_WORDS} # <= because tokenizer is 1 indexed
tokenizer.word_index[tokenizer.oov_token] = MAX_NB_WORDS + 1
word_index = tokenizer.word_index #the dict values start from 1 so this is fine with zeropadding
index2word = {v: k for k, v in word_index.items()}
sequences = tokenizer.texts_to_sequences(texts)
data_1 = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', data_1.shape)
NB_WORDS = (min(tokenizer.num_words, len(word_index))+1) #+1 for zero padding 

Found 20 unique tokens
Shape of data tensor: (10000, 24)


In [10]:
#==================== sample train/validation data =====================#
data_val = data_1[8000:10000]
data_train = data_1[:8000]

In [11]:
#======================== prepare GLOVE embeddings =============================#
embeddings_index = {}
f = open(GLOVE_EMBEDDING, encoding='utf8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Found %s word vectors.' % len(embeddings_index))

glove_embedding_matrix = np.zeros((NB_WORDS, EMBEDDING_DIM))
for word, i in word_index.items():
    if i < NB_WORDS+1: #+1 for 'unk' oov token
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            glove_embedding_matrix[i] = embedding_vector
        else:
            # words not found in embedding index will the word embedding of unk
            glove_embedding_matrix[i] = embeddings_index.get('unk')
print('Null word embeddings: %d' % np.sum(np.sum(glove_embedding_matrix, axis=1) == 0))

Found 400000 word vectors.
Null word embeddings: 2


In [12]:
#====================== VAE model ============================================#
batch_size = 100
max_len = MAX_SEQUENCE_LENGTH
emb_dim = EMBEDDING_DIM
latent_dim = 64
intermediate_dim = 256
epsilon_std = 1.0
kl_weight = 0.01
num_sampled=500
act = ELU()

In [13]:
x = Input(shape=(max_len,))
x_embed = Embedding(NB_WORDS, emb_dim, weights=[glove_embedding_matrix],
                            input_length=max_len, trainable=False)(x)
h = Bidirectional(LSTM(intermediate_dim, return_sequences=False, recurrent_dropout=0.2), merge_mode='concat')(x_embed)
#h = Bidirectional(LSTM(intermediate_dim, return_sequences=False), merge_mode='concat')(h)
#h = Dropout(0.2)(h)
#h = Dense(intermediate_dim, activation='linear')(h)
#h = act(h)
#h = Dropout(0.2)(h)
z_mean = Dense(latent_dim)(h)
z_log_var = Dense(latent_dim)(h)

In [14]:
def sampling(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=(batch_size, latent_dim), mean=0.,
                              stddev=epsilon_std)
    return z_mean + K.exp(z_log_var / 2) * epsilon

# note that "output_shape" isn't necessary with the TensorFlow backend
z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])
# we instantiate these layers separately so as to reuse them later
repeated_context = RepeatVector(max_len)
decoder_h = LSTM(intermediate_dim, return_sequences=True, recurrent_dropout=0.2)
decoder_mean = Dense(NB_WORDS, activation='linear')#softmax is applied in the seq2seqloss by tf #TimeDistributed()
h_decoded = decoder_h(repeated_context(z))
x_decoded_mean = decoder_mean(h_decoded)

In [15]:
# placeholder loss
def zero_loss(y_true, y_pred):
    return K.zeros_like(y_pred)
#Sampled softmax
#logits = tf.constant(np.random.randn(batch_size, max_len, NB_WORDS), tf.float32)
#targets = tf.constant(np.random.randint(NB_WORDS, size=(batch_size, max_len)), tf.int32)
#proj_w = tf.constant(np.random.randn(NB_WORDS, NB_WORDS), tf.float32)
#proj_b = tf.constant(np.zeros(NB_WORDS), tf.float32)
#
#def _sampled_loss(labels, logits):
#    labels = tf.cast(labels, tf.int64)
#    labels = tf.reshape(labels, [-1, 1])
#    logits = tf.cast(logits, tf.float32)
#    return tf.cast(
#                    tf.nn.sampled_softmax_loss(
#                        proj_w,
#                        proj_b,
#                        labels,
#                        logits,
#                        num_sampled=num_sampled,
#                        num_classes=NB_WORDS),
#                    tf.float32)
#softmax_loss_f = _sampled_loss

In [16]:
class CustomVariationalLayer(Layer):
    def __init__(self, kl_weight=1.0, **kwargs):
        super(CustomVariationalLayer, self).__init__(**kwargs)
        self.kl_weight = kl_weight

    def vae_loss(self, x, x_decoded_mean, z_mean, z_log_var):
        xent_loss = tf.reduce_sum(tf.keras.losses.binary_crossentropy(x, x_decoded_mean), axis=-1)
        kl_loss = -0.5 * tf.reduce_sum(1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var), axis=-1)
        return K.mean(xent_loss + self.kl_weight * kl_loss)

    def call(self, inputs):
        x = inputs[0]
        x_decoded_mean = inputs[1]
        z_mean = inputs[2]
        z_log_var = inputs[3]
        
        loss = self.vae_loss(x, x_decoded_mean, z_mean, z_log_var)
        self.add_loss(loss)
        
        # We don't use this output, but it has to have the correct shape:
        return K.ones_like(x)

In [17]:
# # Custom loss layer
# class CustomVariationalLayer(Layer):
#     def __init__(self, **kwargs):
#         self.is_placeholder = True
#         super(CustomVariationalLayer, self).__init__(**kwargs)
#         self.target_weights = tf.constant(np.ones((batch_size, max_len)), tf.float32)

#     def vae_loss(self, x, x_decoded_mean):
#         #xent_loss = K.sum(metrics.categorical_crossentropy(x, x_decoded_mean), axis=-1)
#         labels = tf.cast(x, tf.int32)
#         xent_loss = K.sum(tf.contrib.seq2seq.sequence_loss(x_decoded_mean, labels, 
#                                                      weights=self.target_weights,
#                                                      average_across_timesteps=False,
#                                                      average_across_batch=False), axis=-1)#,
#                                                      #softmax_loss_function=softmax_loss_f), axis=-1)#,
#         kl_loss = - 0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
#         xent_loss = K.mean(xent_loss)
#         kl_loss = K.mean(kl_loss)
#         return K.mean(xent_loss + kl_weight * kl_loss)

#     def call(self, inputs):
#         x = inputs[0]
#         x_decoded_mean = inputs[1]
#         print(x.shape, x_decoded_mean.shape)
#         loss = self.vae_loss(x, x_decoded_mean)
#         self.add_loss(loss, inputs=inputs)
#         # we don't use this output, but it has to have the correct shape:
#         return K.ones_like(x)
    
def kl_loss(x, x_decoded_mean):
    kl_loss = - 0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
    kl_loss = kl_weight * kl_loss
    return kl_loss

In [18]:
loss_layer = CustomVariationalLayer()([x, x_decoded_mean])
vae = Model(x, [loss_layer])
opt = Adam(lr=0.01) 
vae.compile(optimizer='adam', loss=[zero_loss], metrics=[kl_loss])
vae.summary()

StagingError: Exception encountered when calling layer "custom_variational_layer" (type CustomVariationalLayer).

in user code:

    File "C:\Users\Nikla\AppData\Local\Temp\ipykernel_15116\3206165261.py", line 14, in call  *
        z_mean = inputs[2]

    IndexError: list index out of range


Call arguments received by layer "custom_variational_layer" (type CustomVariationalLayer):
  • inputs=['tf.Tensor(shape=(None, 24), dtype=float32)', 'tf.Tensor(shape=(100, 24, 21), dtype=float32)']

In [ ]:
#======================= Model training ==============================#
def create_model_checkpoint(dir, model_name):
    filepath = dir + '/' + model_name + ".h5" 
    directory = os.path.dirname(filepath)
    try:
        os.stat(directory)
    except:
        os.mkdir(directory)
    checkpointer = ModelCheckpoint(filepath=filepath, verbose=1, save_best_only=True)
    return checkpointer

checkpointer = create_model_checkpoint('models', 'vae_seq2seq_test_very_high_std')



vae.fit(data_train, data_train,
     shuffle=True,
     epochs=100,
     batch_size=batch_size,
     validation_data=(data_val, data_val), callbacks=[checkpointer])

print(K.eval(vae.optimizer.lr))
K.set_value(vae.optimizer.lr, 0.01)

#batch_size=512
#nb_epoch=100
#n_steps = 400000/batch_size#404000/batch_size
#for counter in range(nb_epoch):
#    print('-------epoch: ',counter,'--------')
#    vae.fit_generator(sent_generator(TRAIN_DATA_FILE, batch_size/2),
#                          steps_per_epoch=n_steps, epochs=1, callbacks=[checkpointer],
#                          validation_data=(data_1_val, data_1_val))


vae.save('models/vae_lstm.h5')
#vae.load_weights('models/vae_seq2seq_test.h5')
# build a model to project inputs on the latent space
encoder = Model(x, z_mean)
#encoder.save('models/encoder32dim512hid30kvocab_loss29_val34.h5')

# build a generator that can sample from the learned distribution
decoder_input = Input(shape=(latent_dim,))
_h_decoded = decoder_h(repeated_context(decoder_input))
_x_decoded_mean = decoder_mean(_h_decoded)
_x_decoded_mean = Activation('softmax')(_x_decoded_mean)
generator = Model(decoder_input, _x_decoded_mean)


index2word = {v: k for k, v in word_index.items()}
index2word[0] = 'pad'

#test on a validation sentence
sent_idx = 100
sent_encoded = encoder.predict(data_val[sent_idx:sent_idx+2,:])
x_test_reconstructed = generator.predict(sent_encoded, batch_size = 1)
reconstructed_indexes = np.apply_along_axis(np.argmax, 1, x_test_reconstructed[0])
np.apply_along_axis(np.max, 1, x_test_reconstructed[0])
np.max(np.apply_along_axis(np.max, 1, x_test_reconstructed[0]))
word_list = list(np.vectorize(index2word.get)(reconstructed_indexes))
print(' '.join(word_list))
original_sent = list(np.vectorize(index2word.get)(data_val[sent_idx]))
print(' '.join(original_sent))



#=================== Sentence processing and interpolation ======================#
# function to parse a sentence
def sent_parse(sentence, mat_shape):
    sequence = tokenizer.texts_to_sequences(sentence)
    padded_sent = pad_sequences(sequence, maxlen=MAX_SEQUENCE_LENGTH)
    return padded_sent#[padded_sent, sent_one_hot]


# input: encoded sentence vector
# output: encoded sentence vector in dataset with highest cosine similarity
def find_similar_encoding(sent_vect):
    all_cosine = []
    for sent in sent_encoded:
        result = 1 - spatial.distance.cosine(sent_vect, sent)
        all_cosine.append(result)
    data_array = np.array(all_cosine)
    maximum = data_array.argsort()[-3:][::-1][1]
    new_vec = sent_encoded[maximum]
    return new_vec


# input: two points, integer n
# output: n equidistant points on the line between the input points (inclusive)
def shortest_homology(point_one, point_two, num):
    dist_vec = point_two - point_one
    sample = np.linspace(0, 1, num, endpoint = True)
    hom_sample = []
    for s in sample:
        hom_sample.append(point_one + s * dist_vec)
    return hom_sample



# input: original dimension sentence vector
# output: sentence text
def print_latent_sentence(sent_vect):
    sent_vect = np.reshape(sent_vect,[1,latent_dim])
    sent_reconstructed = generator.predict(sent_vect)
    sent_reconstructed = np.reshape(sent_reconstructed,[max_len,NB_WORDS])
    reconstructed_indexes = np.apply_along_axis(np.argmax, 1, sent_reconstructed)
    word_list = list(np.vectorize(index2word.get)(reconstructed_indexes))
    w_list = [w for w in word_list if w not in ['pad']]
    print(' '.join(w_list))
    #print(word_list)
    
       
        
def new_sents_interp(sent1, sent2, n):
    tok_sent1 = sent_parse(sent1, [27])
    tok_sent2 = sent_parse(sent2, [27])
    enc_sent1 = encoder.predict(tok_sent1, batch_size = 16)
    enc_sent2 = encoder.predict(tok_sent2, batch_size = 16)
    test_hom = shortest_homology(enc_sent1, enc_sent2, n)
    for point in test_hom:
        print_latent_sentence(point)
        


#====================== Example ====================================#
sentence1=['gogogo where can i find a bad restaurant endend']
mysent = sent_parse(sentence1, [27])
mysent_encoded = encoder.predict(mysent, batch_size = 16)
print_latent_sentence(mysent_encoded)
print_latent_sentence(find_similar_encoding(mysent_encoded))

sentence2=['gogogo where can i find an extremely good restaurant endend']
mysent2 = sent_parse(sentence2, [27])
mysent_encoded2 = encoder.predict(mysent2, batch_size = 16)
print_latent_sentence(mysent_encoded2)
print_latent_sentence(find_similar_encoding(mysent_encoded2))
print('-----------------')

new_sents_interp(sentence1, sentence2, 5)

